**This notebook is an exercise in the [Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/interactive-maps).**

---


# Introduction

당신은 일본의 도시 안전 계획자이며 일본의 어느 지역에서 추가 지진 보강이 필요한지 분석하고 있습니다. 인구 밀도가 높고 지진이 발생하기 쉬운 지역은 어디입니까?

<중앙>
<img src="https://i.imgur.com/Kuh9gPj.png" width="450"><br/>
</center>

시작하기 전에 아래 코드 셀을 실행하여 모든 것을 설정하세요.

In [ ]:
import pandas as pd
import geopandas as gpd

import folium
from folium import Choropleth
from folium.plugins import HeatMap

from learntools.core import binder
binder.bind(globals())
from learntools.geospatial.ex3 import *

대화형 지도를 표시하기 위해 'embed_map()' 함수를 정의합니다. 지도를 포함하는 변수와 지도가 저장될 HTML 파일의 이름이라는 두 가지 인수를 허용합니다.

이 기능은 지도가 [모든 웹 브라우저에서](https://github.com/python-visualization/folium/issues/812) 표시되도록 합니다.

In [ ]:
def embed_map(m, file_name):
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

### 1) 지진은 판 경계와 일치합니까?

아래 코드 셀을 실행하여 전역 플레이트 경계를 표시하는 DataFrame `plate_boundaries`를 만듭니다. "좌표" 열은 경계를 따라 (위도, 경도) 위치의 목록입니다.

In [ ]:
plate_boundaries = gpd.read_file("../input/geospatial-learn-course-data/Plate_Boundaries/Plate_Boundaries/Plate_Boundaries.shp")
plate_boundaries['coordinates'] = plate_boundaries.apply(lambda x: [(b,a) for (a,b) in list(x.geometry.coords)], axis='columns')
plate_boundaries.drop('geometry', axis=1, inplace=True)

plate_boundaries.head()

다음으로 아래의 코드 셀을 변경 없이 실행하여 과거 지진 데이터를 DataFrame 'earthquakes'에 로드합니다.

In [ ]:
# Load the data and print the first 5 rows
earthquakes = pd.read_csv("../input/geospatial-learn-course-data/earthquakes1970-2014.csv", parse_dates=["DateTime"])
earthquakes.head()

아래 코드 셀은 지도에서 판 경계를 시각화합니다. 모든 지진 데이터를 사용하여 동일한 지도에 히트맵을 추가하여 지진이 판 경계와 일치하는지 확인합니다.

In [ ]:
# Create a base map with plate boundaries
m_1 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_1)

# Your code here: Add a heatmap to the map
HeatMap(data=earthquakes[['Latitude', 'Longitude']], radius=15).add_to(m_1)

# Uncomment to see a hint
#q_1.a.hint()

# Show the map
embed_map(m_1, 'q_1.html')

In [ ]:
# Get credit for your work after you have created a map
#q_1.a.check()

# Uncomment to see our solution (your code may look different!)
#q_1.a.solution()

따라서 위의 지도에서 지진이 판 경계와 일치합니까?
일치합니다.

In [ ]:
# View the solution (Run this code cell to receive credit!)
#q_1.b.solution()

### 2) Is there a relationship between earthquake depth and proximity to a plate boundary in Japan?

최근에 지진의 깊이가 [중요 정보](https://www.usgs.gov/faqs/what-depth-do-earthquakes-occur-what-significance-depth?qt-news_science_products=0#qt)를 알려준다는 내용을 읽었습니다. -news_science_products) 지구의 구조에 대해. 흥미로운 글로벌 패턴이 있는지 확인하고 싶고 일본의 깊이가 어떻게 다른지 이해하고 싶습니다.


In [ ]:
# Create a base map with plate boundaries
m_2 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)
for i in range(len(plate_boundaries)):
    folium.PolyLine(locations=plate_boundaries.coordinates.iloc[i], weight=2, color='black').add_to(m_2)
    
# Your code here: Add a map to visualize earthquake depth
def color_producer(val):
    if val < 50:
        return 'forestgreen'
    elif val < 100:
        return 'darkorange'
    else:
        return 'darkred'

# Uncomment to see a hint
#q_2.a.hint()

# View the map
embed_map(m_2, 'q_2.html')

In [ ]:
# Get credit for your work after you have created a map
#q_2.a.check()

# Uncomment to see our solution (your code may look different!)
#q_2.a.solution()

일본에서 판 경계와의 근접성과 지진 깊이 사이의 관계를 감지할 수 있습니까? 이 패턴이 전 세계적으로 유지됩니까?

일본 북부에서는 판 경계에 가까운 지진이 더 얕아지는 경향이 있는 것으로 보입니다(판 경계에서 먼 지진은 더 깊습니다). 이 패턴은 남미 서부 해안과 같은 다른 지역에서도 반복됩니다. 그러나 모든 곳에서 적용되는 것은 아닙니다(예: 중국, 몽골, 러시아).

In [ ]:
# View the solution (Run this code cell to receive credit!)
#q_2.b.solution()

### 3) Which prefectures have high population density?

다음 코드 셀을 변경 없이 실행하여 일본 현의 지리적 경계를 포함하는 GeoDataFrame '현'을 만듭니다.

In [ ]:
# GeoDataFrame with prefecture boundaries
prefectures = gpd.read_file("../input/geospatial-learn-course-data/japan-prefecture-boundaries/japan-prefecture-boundaries/japan-prefecture-boundaries.shp")
prefectures.set_index('prefecture', inplace=True)
prefectures.head()

다음 코드 셀은 일본 각 현에 대한 인구, 면적(제곱 킬로미터) 및 인구 밀도(제곱 킬로미터당)를 포함하는 DataFrame '통계'를 만듭니다. 변경 없이 코드 셀을 실행합니다.

In [ ]:
# DataFrame containing population of each prefecture
population = pd.read_csv("../input/geospatial-learn-course-data/japan-prefecture-population.csv")
population.set_index('prefecture', inplace=True)

# Calculate area (in square kilometers) of each prefecture
area_sqkm = pd.Series(prefectures.geometry.to_crs(epsg=32654).area / 10**6, name='area_sqkm')
stats = population.join(area_sqkm)

# Add density (per square kilometer) of each prefecture
stats['density'] = stats["population"] / stats["area_sqkm"]
stats.head()

다음 코드 셀을 사용하여 등치 지도를 만들어 인구 밀도를 시각화합니다.

In [ ]:
# Create a base map
m_3 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Your code here: create a choropleth map to visualize population density
Choropleth(geo_data=prefectures['geometry'].__geo_interface__,
           data=stats['density'],
           key_on="feature.id",
           fill_color='YlGnBu',
           legend_name='Population density (per square kilometer)'
          ).add_to(m_3)

# Uncomment to see a hint
#q_3.a.hint()

# View the map
embed_map(m_3, 'q_3.html')

In [ ]:
# Get credit for your work after you have created a map
#q_3.a.check()

# Uncomment to see our solution (your code may look different!)
#q_3.a.solution()

어느 3개 현이 다른 현보다 밀도가 더 높습니까? 그들은 전국에 퍼져 있습니까, 아니면 모두 거의 같은 지리적 지역에 있습니까? (*일본 지리에 익숙하지 않은 경우 [이 지도](https://en.wikipedia.org/wiki/Prefectures_of_Japan)가 질문에 답하는 데 유용할 수 있습니다.)*
##### 도쿄, 가나가와, 오사카는 인구 밀도가 가장 높습니다. 이 도도부현은 모두 일본 중부에 위치하고 있으며 도쿄와 가나가와현이 인접해 있습니다.

In [ ]:
# View the solution (Run this code cell to receive credit!)
#q_3.b.solution()

### 4) 규모가 큰 지진이 발생하기 쉬운 고밀도 현은?

지진 보강의 혜택을 받을 수 있는 한 현을 제안하는 지도를 만듭니다. 지도는 밀도와 지진 규모를 모두 시각화해야 합니다

In [ ]:
# Create a base map
m_4 = folium.Map(location=[35,136], tiles='cartodbpositron', zoom_start=5)

# Your code here: create a map
def color_producer(magnitude):
    if magnitude > 6.5:
        return 'red'
    else:
        return 'green'

Choropleth(
    geo_data=prefectures['geometry'].__geo_interface__,
    data=stats['density'],
    key_on="feature.id",
    fill_color='BuPu',
    legend_name='Population density (per square kilometer)').add_to(m_4)

for i in range(0,len(earthquakes)):
    folium.Circle(
        location=[earthquakes.iloc[i]['Latitude'], earthquakes.iloc[i]['Longitude']],
        popup=("{} ({})").format(
            earthquakes.iloc[i]['Magnitude'],
            earthquakes.iloc[i]['DateTime'].year),
        radius=earthquakes.iloc[i]['Magnitude']**5.5,
        color=color_producer(earthquakes.iloc[i]['Magnitude'])).add_to(m_4)

# Uncomment to see a hint
#q_4.a.hint()

# View the map
embed_map(m_4, 'q_4.html')

In [ ]:
# Get credit for your work after you have created a map
#q_4.a.check()

# Uncomment to see our solution (your code may look different!)
#q_4.a.solution()

추가 지진 보강을 위해 어느 현을 추천합니까?
##### 도쿄는 단연 가장 인구 밀도가 높은 현이며 많은 지진을 경험했습니다. 오사카는 상대적으로 인구 밀도가 낮지만 도쿄 근처보다 상대적으로 강한 지진을 경험했습니다. 그리고 가나가와의 긴 해안(높은 밀도와 강한 지진의 역사적 근접성에 더하여)은 추가적인 잠재적 쓰나미 위험에 대해 걱정하게 만들 수 있습니다.

In [ ]:
# View the solution (Run this code cell to receive credit!)
#q_4.b.solution() 

# Keep going

Learn how to convert names of places to geographic coordinates with **[geocoding](https://www.kaggle.com/alexisbcook/manipulating-geospatial-data)**.  You'll also explore special ways to join information from multiple GeoDataFrames.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/geospatial-analysis/discussion) to chat with other learners.*